In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import tqdm

# Loading Data

Data needs to be downloaded and placed in the `data` folder - but it is not necessary as In this notebook I will initially process and partition the data for further ingestion. However, to fully follow the notebook it can be downloaded here: 

https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

In [3]:
! ls data

archive.zip	 hashes.txt   sample_no_agg.csv     text_df_2.feather
copy		 Reviews.csv  similarity_dict.json  text_df_3.feather
database.sqlite  sample.csv   text_df_1.feather     text_df.feather


We are interested in `database.sqlite` I connected with the data source and extracted all needed information 

In [4]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("data/database.sqlite")

cur = con.cursor()

In [5]:
test_query = """
SELECT * FROM Reviews
LIMIT 100
"""

# probably better to get it without text
query_without_text = """
SELECT ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary FROM Reviews
"""

query_just_text = """
SELECT ProductId, UserId, Text FROM Reviews
"""


In [6]:
df_without_text = pd.read_sql_query(query_without_text, con)

In [10]:
df_just_text = pd.read_sql_query(query_just_text, con)

In [7]:
df_without_text.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all"
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy


In [11]:
df_just_text.head()

,ProductId,UserId,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,Great taffy at a great price. There was a wid...


In [14]:
df = df_just_text.copy()

In [11]:
# write dataframes to feather to further upload to github
df.iloc[:len(df)//3].to_feather('data/text_df_1.feather')
df.iloc[len(df)//3:2*len(df)//3].reset_index(drop=True).to_feather('data/text_df_2.feather')
df.iloc[2*len(df)//3:].reset_index(drop=True).to_feather('data/text_df_3.feather')

In [13]:
df_without_text.to_feather('data/no_text_df.feather')

To obtain the main database.sqlite `df_without_text` has to be merged with concatenated `text_df`'s.

# Text data processing

In [17]:
import spacy
import nltk
from nltk.corpus import stopwords
import re 
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    doc = nlp(text)
    
    # lemmatize, remove stopwords, and convert to lowercase
    cleaned_text = ' '.join([token.lemma_.lower() for token in doc if token.text.lower() not in stop_words])
    return cleaned_text

In [14]:
%%time
df_just_text['Clean_text'] = df_just_text['Text'].apply(clean_text)

CPU times: user 1h 45min 16s, sys: 2.15 s, total: 1h 45min 18s
Wall time: 1h 45min 17s


# interpunction and numbers 
iteratively load, process and save the text dataframes

In [26]:
for x in ['data/text_df_1.feather', 'data/text_df_2.feather', 'data/text_df_3.feather']: 
    df = pd.read_feather(x)

    pattern = r'[\d.,!?;:@#&]+|<[^>]*>|https?://\S+|["\'\(\)_/+\\$-]'
    
    df['Clean_text'] = df['Clean_text'].apply(lambda x: re.sub(pattern, '', x)) # re
    df['Clean_text'] = df['Clean_text'].apply(lambda x: re.sub(r'\s+', ' ', x))

    df.to_feather(x)

# Show 

The semi-clean processing will be used in the `Sentiment` notebook, here just to show for the report (it is not saved) 

In [36]:
df = df_just_text.sample(1000)

In [68]:
pattern = r'[\d.,!?;:@#&\*]+|<[^>]*>|https?://\S+|["\'\(\)_/+\\$-]'
pattern_semiclean = r'|<[^>]*>|https?://\S+|["\'\(\)_/+\\$-]'


In [61]:
df['Clean_text'] = df['Text'].apply(clean_text)

In [69]:
df['Clean_text'] = df['Clean_text'].apply(lambda x: re.sub(pattern, '', x)) # re
df['Clean_text'] = df['Clean_text'].apply(lambda x: re.sub(r'\s+', ' ', x))


df['Semiclean_text'] = df['Text'].apply(lambda x: re.sub(pattern_semiclean, '', x))
df['Semiclean_text'] = df['semiclean_text'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [63]:
df.iloc[93]

ProductId                                                B000MXTMZO
UserId                                               A1VXZJVVLQNX9G
Text              If you're undecided between purchasing this mi...
Clean_text        be undecided purchase mild variety < href= gp ...
semiclean_text    If youre undecided between purchasing this mil...
Semiclean_text    If youre undecided between purchasing this mil...
Name: 376972, dtype: object

In [73]:
df.loc[376972, 'Text'][:300]

'If you\'re undecided between purchasing this mild variety and <a href="http://www.amazon.com/gp/product/B000JSM178">Rajah Madras Curry Powder (Hot) - 3.5oz</a> because you don\'t like your curry *too* hot, I\'d suggest starting with this one.  It has a nice flavor and more of a kick than expected - whi'

In [74]:
df.loc[376972, 'Semi_clean_text'][:300]

'If youre undecided between purchasing this mild variety and Rajah Madras Curry Powder Hot 3.5oz because you dont like your curry *too* hot, Id suggest starting with this one. It has a nice flavor and more of a kick than expected which gives me the idea the hot one would be really hot.The cannister l'

In [75]:
df.loc[376972, 'Clean_text'][:300]

'be undecided purchase mild variety rajah madras curry powder hot oz nt like curry hot would suggest start one nice flavor kick expect give idea hot one would really hotthe cannister look like short version baking powder shaker hole top take lid be curry keep last empty mccormick bottle shaker top fi'